# PREPARATIVOS

In [1]:
import pandas as pd
import numpy as np
import re
import os
from funciones import *

In [2]:
FBL3N = leer_excel("FBL3N.xlsx")

Leyendo archivo Excel: FBL3N.xlsx
Archivo Excel leído.


In [3]:
FBL5N = leer_excel("FBL5N.xlsx")

Leyendo archivo Excel: FBL5N.xlsx
Archivo Excel leído.


In [4]:
KNA1 = leer_excel("KNA1.xlsx")

Leyendo archivo Excel: KNA1.xlsx
Archivo Excel leído.


In [5]:
PROPUESTA = leer_excel("PROPUESTA.xlsx", None)

Leyendo archivo Excel: PROPUESTA.xlsx
Archivo Excel leído.


# PARTE 1

In [6]:
# PARTE 1

textos = FBL3N["Texto"]
#FBL3N["Categoría"] = textos.apply(detectar_categoria)
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)
#FBL3N["Números"] = textos.apply(detectar_numeros)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]
#con_otros_numeros = FBL3N[pd.isna(FBL3N["RUT"]) & pd.isna(FBL3N["N° Factura"]) & pd.notna(FBL3N["Números"])]
#sin_numero = FBL3N[pd.isna(FBL3N["RUT"]) & pd.isna(FBL3N["N° Factura"]) & pd.isna(FBl3N["Números"])]

"""
try:
	os.mkdir("FBL3N/")
except FileExistsError:
	pass

for dataframe, nombre in ((FBL3N, "NUEVO"), (con_rut, "RUT"), (con_n_factura, "N_FACTURA"), (con_otros_numeros, "OTROS_N"), (sin_numero, "SIN_N")):
	escribir_excel(dataframe, "FBL3N/" + nombre + ".xlsx")
"""

'\ntry:\n\tos.mkdir("FBL3N/")\nexcept FileExistsError:\n\tpass\n\nfor dataframe, nombre in ((FBL3N, "NUEVO"), (con_rut, "RUT"), (con_n_factura, "N_FACTURA"), (con_otros_numeros, "OTROS_N"), (sin_numero, "SIN_N")):\n\tescribir_excel(dataframe, "FBL3N/" + nombre + ".xlsx")\n'

# PARTE 2

In [7]:
# Calzar números de factura
# ARREGLAR ESTO
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
MATCHES_FACT = pd.merge(FACT_FBL3N, FBL5N, left_on="N° Factura", right_on="Referencia")

In [8]:
MATCHES_FACT

,Asignación,Nº documento,División,Clase de documento,Fecha de documento,Clave contabiliz.,Importe en moneda local,Moneda local,Indicador impuestos,Doc.compensación,...,Importe en ML,CPag,Texto_y,nan,NaT,Clave ref.1,Clave ref.2,VP,Lib.mayor,Usuario
0,2.021101e+12,100370180.0,NaN,CT,2021-10-07,50.0,-18350,CLP,NaN,NaN,...,18350,Z030,NaN,NaN,NaT,Innominado,1745727,F,1104011001,SD060


In [9]:
# Calzar RUTs
FBL5N["Cuenta"] = FBL5N["Cuenta"].apply(lambda n: str(n))
KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"].apply(lambda s: str(s))
ID_FBL3N = pd.merge(RUT_FBL3N, KNA1[["Nº ident.fis.1", "Cliente"]], left_on="RUT", right_on="Nº ident.fis.1")
MATCHES_ID = pd.merge(ID_FBL3N, FBL5N, left_on="Cliente", right_on="Cuenta", how="left")

In [10]:
MATCHES_ID

,Asignación,Nº documento,División,Clase de documento,Fecha de documento,Clave contabiliz.,Importe en moneda local,Moneda local,Indicador impuestos,Doc.compensación,...,Importe en ML,CPag,Texto_y,nan,NaT,Clave ref.1,Clave ref.2,VP,Lib.mayor,Usuario
0,0.000000e+00,100133490.0,NaN,CT,2020-04-06,50.0,-82394,CLP,NaN,NaN,...,-23448.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,GVERGARA
1,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,JFABRES
2,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,NaN,...,-1104.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,JFABRES
3,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,NaN,...,-1.0,NaN,ABONO SEGUN FACTURA 8913231,NaN,NaT,NaN,NaN,NaN,1.104011e+09,VPADRON
4,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,NaN,...,-1.0,NaN,ABONO SEGUN FACTURA 8935682,NaN,NaT,NaN,NaN,NaN,1.104011e+09,VPADRON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27009,2.021102e+12,100382773.0,NaN,CT,2021-10-18,50.0,-47005,CLP,NaN,NaN,...,316914.0,Z030,NaN,NaN,NaT,Nominado,NaN,F,1.104011e+09,SD050
27010,2.021102e+12,100382773.0,NaN,CT,2021-10-18,50.0,-47005,CLP,NaN,NaN,...,178637.0,Z030,NaN,NaN,NaT,Nominado,NaN,F,1.104011e+09,SD050
27011,2.021102e+12,100382773.0,NaN,CT,2021-10-18,50.0,-47005,CLP,NaN,NaN,...,80646.0,Z030,NaN,NaN,NaT,Nominado,NaN,F,1.104011e+09,SD050
27012,2.021102e+12,100382773.0,NaN,CT,2021-10-18,50.0,-47005,CLP,NaN,NaN,...,403827.0,Z030,NaN,NaN,NaT,Nominado,NaN,F,1.104011e+09,SD050


In [11]:
escribir_excel(ID_FBL3N, "ID_FBL3N.xlsx")

Escribiendo nuevo archivo Excel: ID_FBL3N.xlsx
Archivo Excel escrito.


In [12]:
escribir_excel(MATCHES_ID, "MATCHES_ID.xlsx")

Escribiendo nuevo archivo Excel: MATCHES_ID.xlsx
Error: el archivo ya está abierto. Ciérralo y vuelve a intentarlo.
Presiona Enter cuando esté listo.
Escribiendo nuevo archivo Excel: MATCHES_ID.xlsx
Archivo Excel escrito.


In [13]:
escribir_excel(MATCHES_FACT, "MATCHES_FACT.xlsx")

Escribiendo nuevo archivo Excel: MATCHES_FACT.xlsx
Archivo Excel escrito.


# PARTE 3

### Trabajar con calces de ID

In [14]:
filtro_1a1 = MATCHES_ID[MATCHES_ID["Importe en moneda local"] == -MATCHES_ID["Importe en ML"]]

In [15]:
escribir_excel(filtro_1a1, "MATCHES_ID_1-1.xlsx")

Escribiendo nuevo archivo Excel: MATCHES_ID_1-1.xlsx
Error: el archivo ya está abierto. Ciérralo y vuelve a intentarlo.
Presiona Enter cuando esté listo.
Escribiendo nuevo archivo Excel: MATCHES_ID_1-1.xlsx
Archivo Excel escrito.


# PARTE 4

### Determinar qué datos encontré yo que Ricardo no, y viceversa

In [16]:
RUT_PROPUESTA = pd.concat([PROPUESTA["Rut y Monto"], PROPUESTA["Solo Rut"]])

In [17]:
outer = pd.merge(filtro_1a1, RUT_PROPUESTA, on="Nº documento", how="outer", indicator=True)

In [18]:
escribir_excel(outer[outer["_merge"] == "left_only"], "LEFT.xlsx")

Escribiendo nuevo archivo Excel: LEFT.xlsx
Error: el archivo ya está abierto. Ciérralo y vuelve a intentarlo.
Presiona Enter cuando esté listo.
Escribiendo nuevo archivo Excel: LEFT.xlsx
Archivo Excel escrito.


In [19]:
escribir_excel(outer[outer["_merge"] == "right_only"], "RIGHT.xlsx")

Escribiendo nuevo archivo Excel: RIGHT.xlsx
Archivo Excel escrito.
